In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':14})
from scipy.integrate import odeint
import pickle

In [2]:
def toggle_switch(y,t): # two state system, with no inputs

    a1 = 0.5;
    a2 = 0.5;
    d1 = 0.5;
    d2 = 0.5;
    n1 = 4; 
    n2 = 2; 
    K1 = 1;
    K2 = 1;

    dy0 = a1/(K1+y[1]**n2) - d1*y[0] 
    dy1 = a1/(K2+y[0]**n1) - d2*y[1]
    
    return[dy0,dy1]

In [4]:
numStates = 2

tStart = 0.0
tEnd = 40
dt = 0.5
numTimepoints = int(round((tEnd-tStart)/dt))
tSpan = np.linspace(tStart,tEnd,numTimepoints)

numTrajectories = 100 # pick a number divisible by 2 for splitting into train and test sets

data = np.zeros(shape=[numStates,numTimepoints,numTrajectories])
for traj in range(0,numTrajectories):
    x0 = np.random.uniform(0.01,1,numStates)
    x = odeint(toggle_switch,x0,tSpan)
    x = x.T[:,:,np.newaxis]
    data[:,:,traj:traj+1] = x
    
    if numTrajectories < 21:
        plt.figure();
        h = plt.plot(tSpan,data[:,:,traj].T);
        plt.legend((h[0],h[1]),('x1','x2'),loc='best');
        plt.xlabel('time');
        plt.ylabel('protein concentration');
# plt.savefig('sample_traj.png')

In [5]:
from copy import deepcopy
data_normed = deepcopy(data)
for i in range(0,numTrajectories):
    data_normed[:,:,i] = (data[:,:,i]-data[:,:,i].mean(axis=1).reshape(numStates,1))/(data[:,:,i].std(axis=1).reshape(numStates,1))

In [6]:
data_normed = data_normed.reshape(numStates,int(numTimepoints*numTrajectories),order='F')
data = data.reshape(numStates,int(numTimepoints*numTrajectories),order='F')
data_normed = (data - data.mean(axis=1).reshape(numStates,1))/data.std(axis=1).reshape(numStates,1)

In [7]:
# pickle.dump([data_normed,numTimepoints,numTrajectories], open('toggle_switch_data_normed.p', 'wb'))
# pickle.dump([data,numTimepoints,numTrajectories], open('toggle_switch_data.p', 'wb'))

In [10]:
Xp = data[:,:-1].T
Xf = data[:,1:].T

save_deepDMD = 0
if save_deepDMD: 
    pickle.dump([Xp,Xf],open('/Users/aqib/Desktop/UCSB/Research/BCCL/darpa-sd2/deepDMD/koopman_data/toggle_switch_sim.p','wb'))